In [1]:
# use CEASELESS Kernel on adrien local laptop

import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # => to add in Seastar env
import seastar as ss

from scipy import interpolate, stats # => to add in Seastar env ?

# from seastar.gmfs.doppler import *

In [2]:
L1 = xr.Dataset(
        data_vars=dict(
            IncidenceAngleImage=(['GroundRange','CrossRange'], np.full([10,10], 30) ),
            LookDirection=(['GroundRange','CrossRange','Antenna'],
                           np.stack( (np.full([10,10], 45),
                                      np.full([10,10], 90),
                                      np.full([10,10], 135)
                                      ), axis=-1
                                   )
                          ),
            Polarization=(['GroundRange','CrossRange'], np.full([10,10], 1) ),
            CentralWavenumber=( [],270 ),
            CentralFreq=( [], 13.5 * 10**9 ),
        ),
        coords=dict(
            GroundRange=np.arange(0,10),
            CrossRange=np.arange(0,10),
            Antenna=['Fore','Mid','Aft']
        ),
)
L1.Polarization[2,2] = 2
L1

<xarray.Dataset>
Dimensions:              (Antenna: 3, CrossRange: 10, GroundRange: 10)
Coordinates:
  * GroundRange          (GroundRange) int64 0 1 2 3 4 5 6 7 8 9
  * CrossRange           (CrossRange) int64 0 1 2 3 4 5 6 7 8 9
  * Antenna              (Antenna) <U4 'Fore' 'Mid' 'Aft'
Data variables:
    IncidenceAngleImage  (GroundRange, CrossRange) int64 30 30 30 ... 30 30 30
    LookDirection        (GroundRange, CrossRange, Antenna) int64 45 90 ... 135
    Polarization         (GroundRange, CrossRange) int64 1 1 1 1 1 ... 1 1 1 1 1
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10

In [3]:
aux0 = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['GroundRange','CrossRange'], np.full([10,10], 5) ),
            WindDirection=(['GroundRange','CrossRange'], np.full([10,10], 90) ),
        ),
        coords=dict(
            GroundRange=np.arange(0,10),
            CrossRange=np.arange(0,10),
        ),
)
aux0

<xarray.Dataset>
Dimensions:        (CrossRange: 10, GroundRange: 10)
Coordinates:
  * GroundRange    (GroundRange) int64 0 1 2 3 4 5 6 7 8 9
  * CrossRange     (CrossRange) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    WindSpeed      (GroundRange, CrossRange) int64 5 5 5 5 5 5 5 ... 5 5 5 5 5 5
    WindDirection  (GroundRange, CrossRange) int64 90 90 90 90 ... 90 90 90 90

In [4]:
aux1 = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['GroundRange','CrossRange'], np.full([10,20], 5) ),
            WindDirection=(['GroundRange','CrossRange'], np.full([10,20], 90) ),
        ),
        coords=dict(
            GroundRange=np.arange(0,10),
            CrossRange=np.arange(0,20),
        ),
)
aux1

<xarray.Dataset>
Dimensions:        (CrossRange: 20, GroundRange: 10)
Coordinates:
  * GroundRange    (GroundRange) int64 0 1 2 3 4 5 6 7 8 9
  * CrossRange     (CrossRange) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
Data variables:
    WindSpeed      (GroundRange, CrossRange) int64 5 5 5 5 5 5 5 ... 5 5 5 5 5 5
    WindDirection  (GroundRange, CrossRange) int64 90 90 90 90 ... 90 90 90 90

In [5]:
aux2 = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['GroundRange','CrossRange'], np.full([10,20], 5) ),
            WindDirection=(['GroundRange','CrossRange'], np.full([10,20], 90) ),
        ),
        coords=dict(
            GroundRange=np.arange(0,10),
            CrossRange=np.arange(5,25),
        ),
)
aux2

<xarray.Dataset>
Dimensions:        (CrossRange: 20, GroundRange: 10)
Coordinates:
  * GroundRange    (GroundRange) int64 0 1 2 3 4 5 6 7 8 9
  * CrossRange     (CrossRange) int64 5 6 7 8 9 10 11 ... 18 19 20 21 22 23 24
Data variables:
    WindSpeed      (GroundRange, CrossRange) int64 5 5 5 5 5 5 5 ... 5 5 5 5 5 5
    WindDirection  (GroundRange, CrossRange) int64 90 90 90 90 ... 90 90 90 90

In [6]:
aux3 = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['GroundRange','CrossRange'], np.full([10,20], 5) ),
            WindDirection=(['GroundRange','CrossRange'], np.full([10,20], 90) ),
        ),
        coords=dict(
            GroundRange=np.arange(0,10),
            CrossRange=np.arange(30,50),
        ),
)
aux3

<xarray.Dataset>
Dimensions:        (CrossRange: 20, GroundRange: 10)
Coordinates:
  * GroundRange    (GroundRange) int64 0 1 2 3 4 5 6 7 8 9
  * CrossRange     (CrossRange) int64 30 31 32 33 34 35 36 ... 44 45 46 47 48 49
Data variables:
    WindSpeed      (GroundRange, CrossRange) int64 5 5 5 5 5 5 5 ... 5 5 5 5 5 5
    WindDirection  (GroundRange, CrossRange) int64 90 90 90 90 ... 90 90 90 90

# Test with L1 and 'aux' the same size, for different antennas 

In [7]:
dop_c = np.full(L1.IncidenceAngleImage.shape, np.nan)

In [8]:
dop_c * L1.CentralFreq.data

array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [9]:
wasv_fore = ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Fore'), aux0, gmf='mouche12')
wasv_fore

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.8688 -0.8688 ... -0.8688

In [10]:
wasv_mid = ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Mid'), aux0, gmf='mouche12')
wasv_mid

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -1.151 -1.151 ... -1.151 -1.151

In [11]:
wasv_aft = ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Aft'), aux0, gmf='mouche12')
wasv_aft

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.8688 -0.8688 ... -0.8688

## Without the precision on the antenna

In [12]:
ss.gmfs.doppler.compute_wasv(L1, aux0, gmf='mouche12')
# without subselection of the Antenna => raise ERROR

Exception: L1.LookDirection need to be a 2D field. 
Use e.g. L1.sel(Antenna="Fore") to reduce to a 2D field.

# Test on Yurovsky + optional keywords

In [13]:
ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Fore'), aux0, gmf='yurovsky19')

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.2449 -0.2449 ... -0.2449

In [14]:
ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Fore'), aux0, gmf='yurovsky19', swh = 1)

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 10)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.4926 -0.4926 ... -0.4926

# Test differences in size between L1 and aux

In [15]:
ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Fore'), aux1, gmf='yurovsky19')

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 20)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.2449 -0.2449 ... nan nan

In [16]:
ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Fore'), aux2, gmf='yurovsky19')

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 25)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 nan nan nan nan ... nan nan nan

In [17]:
ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Fore'), aux3, gmf='yurovsky19')

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 30)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 nan nan nan nan ... nan nan nan

In [18]:
ss.gmfs.doppler.compute_wasv(L1.sel(Antenna='Fore'), aux1, gmf='mouche12')

<xarray.Dataset>
Dimensions:  (CrossRange: 10, GroundRange: 20)
Dimensions without coordinates: CrossRange, GroundRange
Data variables:
    WASV     (CrossRange, GroundRange) float64 -0.8688 -0.8688 ... nan nan